In [1]:
%config Completer.use_jedi = False
%matplotlib widget

In [2]:
import numpy as np
import pandas as pd
import mne

## Config

In [3]:
fname = "../data/20210824-130224.hdf5"
device = "eeg/1"
drop = ["counter", "timestamp", "button", "sync"]

## Load data

In [4]:
store = pd.HDFStore(fname, "r")

In [5]:
for key in store.keys():
    print(key)

/events
/eeg/1


In [14]:
events = store.select("events")
signal = store.select(device)

In [15]:
events

,label,data
2021-08-24 13:03:19.571000000,control_start,NaN
2021-08-24 13:03:32.792800049,control_stop,NaN
2021-08-24 13:03:44.640300049,observation,"""something happened"""
2021-08-24 13:03:59.594699951,meditation_start,
2021-08-24 13:04:13.004699951,meditation_stop,


In [7]:
signal = signal.drop(columns=drop)

## Convert to MNE

### Extract annotations

In [8]:
labels = events['label'].values
onsets = events.index.values.astype(np.int64) * 1e-9
annotations = mne.Annotations(onsets, 0, labels, orig_time=0)
annotations

<Annotations | 5 segments: control_start (1), control_stop (1), ...>

### Extract signal info

In [9]:
channels = list(signal.columns.values)
times = signal.index.values.astype(np.int64) * 1e-9
data = signal.values.T * 1e-6
rate = store.get_node(device)._v_attrs['meta']['rate']

### Create MNE object

In [10]:
info = mne.create_info(ch_names=channels, sfreq=rate, ch_types='eeg')
info.set_montage('standard_1020')
raw = mne.io.RawArray(data, info)
raw._filenames = [fname] # Hotfix: see https://github.com/mne-tools/mne-python/issues/9385
raw.set_meas_date(times[0])
raw.set_annotations(annotations)

<RawArray | 20210824-130224.hdf5, 18 x 66596 (130.1 s), ~9.2 MB, data loaded>

## Filtering

In [11]:
raw.notch_filter(50, method="iir")
raw.filter(1, 40, method="iir")

<RawArray | 20210824-130224.hdf5, 18 x 66596 (130.1 s), ~9.2 MB, data loaded>

## Vizualization

### Raw signal

In [12]:
fig = raw.plot() 

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Power spectrum

In [13]:
fig = raw.plot_psd(fmax=40)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …